<a href="https://colab.research.google.com/github/NahyeonKang/classify-spending/blob/main/build_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. 음식점
1. 카페
2. 식재료
3. 편의점
5. 뷰티
6. 패션
7. 의료
8. 여행
9. 공연/영화
10. 스포츠
11. 기타 쇼핑
12. 간편결제
13. 교육비
14. 교통비
15. 생활요금
16. 기타


In [ ]:
import pandas as pd

data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/data.xlsx", header=None)

In [ ]:
data.columns=['spending']

In [ ]:
data['category'] = "0.restaurant"

In [ ]:
data

,spending,category
0,옛집국수,0. restaurant
1,롯데백화점 본점,0. restaurant
2,우리마키 성수점,0. restaurant
3,테이큰 커피,0. restaurant
4,(주)범화 청송(청주)휴게소,0. restaurant
...,...,...
659,금고ATM입금,0. restaurant
660,가시버시갈비탕.찜,0. restaurant
661,펀비어킹,0. restaurant
662,한국철도공사,0. restaurant


# GPT4 API로 분류하기

In [ ]:
! pip install openai

In [ ]:
import openai

openai.api_key = ""

In [ ]:
for i in range(166, 664):
  spending = data.loc[i]['spending']

  messages = [
      {"role":"system", "content": "Categorize a given <spending> into the following categories. \n0.restaurant\n1.cafe\n2.grocery\n3.convenience store\n4.cosmetics\n5.fashion\n6.healthcare\n7.travel\n8.shows/movies\n9.sports\n10.other shoppings\n11.payment service\n12.education\n13.transportation\n14.utility bills\n15.Others  "},
      {"role":"user", "content":f"<spending>\n{spending}"}
  ]

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
  )

  answer = response['choices'][0]['message']['content']
  data.loc[i]['category']=answer

APIError: ignored

In [ ]:
i

213

In [ ]:
data.to_excel("/content/drive/MyDrive/Colab Notebooks/GPTlabeleddata.xlsx")

# GPT4 답변 & 크롤링 데이터로 huggingface Dataset 만들기

In [ ]:
dataset = load_dataset('csv', data_files={'train': "train_set.csv",'test': "test_set.csv"})

# BERT 학습하기

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install accelerate -U

In [ ]:
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed=42)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

nsmc = load_dataset("nsmc")

In [ ]:
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
nsmc['train'][0]

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}

## num_labels 추가

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=15).to('cuda')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["document"], truncation=True)

In [ ]:
tokenized_nsmc = nsmc.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
import torch
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="ai.keepit/classifier",
    learning_rate=5e-6,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_nsmc["train"],
    eval_dataset=tokenized_nsmc["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/nile/ai.keepit into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.271500,0.260387,0.895700
2,0.213700,0.267716,0.900260
3,0.165500,0.304559,0.902040


TrainOutput(global_step=28125, training_loss=0.233152838406033, metrics={'train_runtime': 5655.3148, 'train_samples_per_second': 79.571, 'train_steps_per_second': 4.973, 'total_flos': 1.25462040082272e+16, 'train_loss': 0.233152838406033, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/416M [00:00<?, ?B/s]

Upload file runs/Aug03_16-21-00_547308ba3464/events.out.tfevents.1691079666.547308ba3464.8391.0:   0%|        …

To https://huggingface.co/nile/ai.keepit
   a03536e..5c6c240  main -> main

   a03536e..5c6c240  main -> main

To https://huggingface.co/nile/ai.keepit
   5c6c240..0051261  main -> main

   5c6c240..0051261  main -> main



'https://huggingface.co/nile/ai.keepit/commit/5c6c2401cf07d82e20a94eb5e228739aca5b42c8'